In [208]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import time 
import re 
from tqdm.notebook import tqdm

import shutil #이미지저장

### get urls

In [180]:
#크롬드라이버 작동
#test계정 punchless86@gmail.com/k145789!
user = 'punchless86@gmail.com'
password = 'k145789!'
keyword = "흑당버블티" 
img_save = True
url = "https://www.instagram.com/explore/tags/{}/".format(keyword) 
# 크롬드라이버 작동 
options.add_argument('--start-maximized') #전체화면
options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.get(url) 
time.sleep(3) 

In [ ]:
#로그인(id, password 입력)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
ele.send_keys(user)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
ele.send_keys(password)
ele = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button > div')
ele.click()
time.sleep(1)
ele = driver.find_element_by_css_selector('#react-root > section > main > div > div > div > div > button') #로그인저장 창 통제
ele.click()

In [508]:
nonetype = type(print())
index_dict = dict()
if img_save == True:
    os.makedirs('./img',exist_ok=True) #이미지_저장폴더

for index, url_half in tqdm(enumerate(url_halfs)):
    #저장 초기화
    contents = dict()
    contents['body'] = []
    contents['hashtags'] = []
    contents['comments'] = []
    contents['like'] = []
    contents['movie'] = []
    contents['url'] = []
    contents['image'] =[]
    real_url = f'https://www.instagram.com/{url_half}'
    driver.get(real_url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    #dates
    try :
        dates = soup.select('div.k_Q0X.NnvRN')[0].text
    except :
        dates = ''
    
    #본문
    try :
        body = soup.select_one('body div.C4VMK > span').text
    except : 
        body = []
    
    #comment
    try : 
        comments = soup.select('div.C4VMK > span')
        comments = [comment.text for comment in comments]
    except:
        comments = []
        
    #해쉬태그
    try :
        hashtags = soup.find_all(attrs={'class':'xil3i'})
        hashtags = [hashtag.text for hashtag in hashtags]
    except :
        print('no hashtag')
        hashtags.append([])

    #좋아요수(동영상의 경우 view) #코드 요약 필요할듯 
    p_tag = soup.select_one('div.Nm9Fw') #사진테그
    m_tag = soup.select_one('span.vcOH2') #동영상테그
    if type(m_tag) == nonetype: #사진인경우
#         print('p_tag')
        try:
            movie=0
            like = soup.select_one('div.Nm9Fw span').text #좋아요 수         
        except :
            like = 0
    elif type(p_tag) == nonetype:#동영상인경우
#         print('m_tag')
        try :
            movie=1
            like = soup.select_one('div.vcOH2 span').text #조회수 수         
        except :
            like = 0
    #image저장 (현재 사진이 여러장인경우 가져오지 못함-->??? 되네???--> 안되네?-->복불복?)
    if img_save == True:
        try :
            imgurl = soup.select('div.KL4Bh')[0].img['src']
            response = requests.get(imgurl, stream = True)
            local_file = open(f'./img/{keyword}_{index}.jpg','wb') #저장임시파일
            response.raw.decode_content = True #역할이...?
            shutil.copyfileobj(response.raw, local_file)
            print(f'{index}, 이미지저장')
            image = 1
        except Exception as ex:
            print(ex)
            print(f'{index}, 이미지저장실패')
            image = 0
            pass
#     else :
#         contents['like'].append('0') 
#     try :
#         try: #좋아요 0인경우
#             soup.select_one('div.Nm9Fw').text
#             like = soup.select_one('div.Nm9Fw span').text
#             contents['movie'].append('X')
#         except :
#             contents['like'].append('0')   
        
#     except:
#         try : #조회수 0인경우
#             like = soup.select_one('span.vcOH2').text
#             like = soup.select_one('span.vcOH2 span').text
#             contents['movie'].append("O")
#         except :
#             contents['like'].append('0') 

        
    #저장
    contents['body'].append(body)
    contents['comments'].append(comments)
    contents['hashtags'].append(hashtags)
    contents['like'].append(like)   
    contents['movie'].append(movie)
    contents['url'].append(real_url)
    contents['image'].append(image)

    #인덱스맞춰저장
    index_dict[index] = contents

In [510]:
temp = pd.DataFrame(index_dict)
temp = temp.T

### 일부사진저장문제

In [520]:
temp['url'].iloc[-1][0]

'https://www.instagram.com//p/CHHpmBvr6Sl/'

### 작업본

In [493]:
temp['url'].iloc[11][0]

'https://www.instagram.com//p/CG9UCCdHUM1/'

In [524]:
# url = 'https://www.instagram.com//p/CGsCvQVnpPt/' #영상 샘픔
url = temp['url'].iloc[11][0]
url = 'https://www.instagram.com//p/CG1_tv2p7sz/'
url = temp['url'].iloc[-2][0]
driver.get(url)
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [487]:
real_url
driver.get(real_url)
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [ ]:
#react-root > section > main > div > div > article > div.eo2As > section.EDfFK.ygqzn > div > div > button
#react-root > section > main > div > div.ltEKP > article > div.eo2As > section.EDfFK.ygqzn > div > span > span #동영상

In [537]:
url = urls.iloc[1][0]
driver.get(url)
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
try :
    imgurl = soup.select('div.KL4Bh')[0].img['src']
    response = requests.get(imgurl, stream = True)
    local_file = open(f'./img/{keyword}_{index}.jpg','wb') #저장임시파일
    response.raw.decode_content = True #역할이...?
    shutil.copyfileobj(response.raw, local_file)
    print(f'{index}, 이미지저장')
    image = 1
except Exception as ex:
    print(ex)
    print(f'{index}, 이미지저장실패')
    image = 0
    pass

716, 이미지저장


In [540]:
urls = temp['url'].iloc[-30:-10]

In [541]:
for url in urls:
    url = url[0]
    driver.get(url)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    try :
        imgurl = soup.select('div.KL4Bh')[0].img['src']
        response = requests.get(imgurl, stream = True)
        local_file = open(f'./img/{keyword}_{index}.jpg','wb') #저장임시파일
        response.raw.decode_content = True #역할이...?
        shutil.copyfileobj(response.raw, local_file)
        print(f'{index}, 이미지저장')
        image = 1
    except Exception as ex:
        print(ex)
        print(f'{index}, 이미지저장실패')
        image = 0
        pass

716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장
716, 이미지저장


In [407]:
#dates
dates = soup.select('div.k_Q0X.NnvRN')[0].text
dates

'1일 전'

In [375]:
#comment
comments = soup.select('div.C4VMK > span')
comments = [comment.text for comment in comments]

In [283]:
a = soup.select_one('div.eo2As div.Nm9Fw')

In [287]:
a = soup.select_one('div.eo2As div.vcOH2')

In [288]:
len(a)

TypeError: object of type 'NoneType' has no len()

In [275]:
nonetype = type(print())

In [276]:
nonetype

NoneType

In [262]:
soup.select_one('div.Nm9Fw')

<div class="Nm9Fw">가장 먼저 <button class="sqdOP yWX7d _8A5w5 " type="button">좋아요</button>를 눌러보세요</div>

In [230]:
temp = pd.DataFrame(contents)
temp['like'] = temp['like'].apply(lambda x : int(x.replace(',','')))

In [235]:
temp[temp['movie'] == 'O']['url']


30    https://www.instagram.com//p/CGsCvQVnpPt/
64    https://www.instagram.com//p/CHT7l9FBJ_m/
Name: url, dtype: object

In [232]:
temp.sort_values('like',ascending = False)

,body,hashtags,like,movie,url
4,• 흑당 버블 밀크티 • #광고 #협찬⠀티바이...,"[#광고, #협찬]",5164,X,https://www.instagram.com//p/CHID0hUsRwM/
24,Brown Sugar <Deerioca Fever with Fresh Milk> @...,[],563,X,https://www.instagram.com//p/CHG0O-JHJnH/
20,#일요일오후 #딸램과데이트 #여자들의수다 #여주커피숍 #단골커피숍 #여주커피나무 #...,"[#일요일오후, #딸램과데이트, #여자들의수다, #여주커피숍, #단골커피숍, #여주...",299,X,https://www.instagram.com//p/CHVHhECnBo2/
6,마시그레이 차한잔🍵말차 흑당버블티 해피주말되세요^^,[],198,X,https://www.instagram.com//p/CHCQHZgHKew/
30,흑당 버블티🖤어서와~ 우리집 홈카페는 처음이지~?☺️,[],192,O,https://www.instagram.com//p/CGsCvQVnpPt/
...,...,...,...,...,...
52,#2021년 #신축년 #1월6일 #찐추위❄ #써니가든🌻#영업중입니다 #동네골목길#동...,"[#2021년, #신축년, #1월6일, #찐추위❄, #써니가든🌻, #영업중입니다, ...",6,X,https://www.instagram.com//p/B-RdJAxjHN8/
26,요거프레소 ☕️ #흑당버블티 #요거프레소 #카페 #09,"[#흑당버블티, #요거프레소, #카페, #09]",6,X,https://www.instagram.com//p/CHCQr7aBwgS/
50,민트색의 예쁜분위기쩐주단 송도점이 짱이에요!저희 쩐주단 송도점은항상 고객님께서기분좋...,"[#흑당, #차, #건강, #버블티, #송도카페, #버블티맛집, #송도버블티맛집, ...",5,X,https://www.instagram.com//p/CGjxMdll-_8/
35,#타이거슈가 #대구타이거슈가 #타이거슈가 #타이거슈가흑당밀크티 #흑당 #흑당버블티 ...,"[#타이거슈가, #대구타이거슈가, #타이거슈가, #타이거슈가흑당밀크티, #흑당, #...",4,X,https://www.instagram.com//p/CHUswV0jk3b/


In [222]:
soup.select_one('span.vcOH2 span').text

'192'

In [ ]:
<span>192</span>

In [ ]:
#react-root > section > main > div > div.ltEKP > article > div.eo2As > section.EDfFK.ygqzn > div > div > button

In [ ]:
#react-root > section > main > div > div.ltEKP > article > div.eo2As > section.EDfFK.ygqzn > div > span

In [ ]:
#react-root > section > main > div > div.ltEKP > article > div.eo2As > section.EDfFK.ygqzn > div > div > button > span

In [158]:
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [159]:
url_source = soup.select('#react-root > section > main > article  a')
url_halfs = []
for url_half in url_source:
    url_halfs.append(url_half.attrs['href'])
    

In [99]:
real_url = f'https://www.instagram.com/{url_half}'

In [102]:
driver.get(real_url)

In [104]:
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [115]:
#저장
contents = dict()
contents['body'] = []
contents['hashtags'] = []
contents['like'] = []

#본문
body = soup.select_one('body div.C4VMK > span').text

#해쉬태그
try :
    hashtags = soup.find_all(attrs={'class':'xil3i'})
    hashtags = [hashtag.text for hashtag in hashtags]
except :
    print('no hashtag')
    hashtags.append([])

#좋아요수
like = soup.select_one('div.Nm9Fw span').text

contents['body'].append(body)
contents['hashtags'].append(hashtags)
contents['like'].append(like)

'• 흑당 버블 밀크티 • #광고 #협찬⠀티바인 30초 버블티 세트로 만든 세번째 레시피!버블에 흑당이 함유되어 있는데, 많을수록 좋잖아요? :)이번에도 30초안에 호다닥 만들 수 있는 초간단 레시피!!!아이들 입맛에도 찰떡, 어른들 입맛에도 찰떡입니다 💖몇몇분들께 샘플로 드렸는데 너무 맛있다고 그러시네요😆🙏⠀티바인 30초 버블티 만들기 세트 공구 프로필링크에서 진행중⠀⠀'

In [ ]:
body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul > ul:nth-child(3) > div > li > div > div.C7I1f > div.C4VMK > span

In [77]:
#해쉬태그
try :
    hashtags = soup.find_all(attrs={'class':'xil3i'})
    hashtags = [hashtag.text for hashtag in hashtags]
except :
    print('no hashtag')
    hashtags.append([])

In [62]:
#좋아요수
like = soup.select_one('div.Nm9Fw span').text

In [ ]:
#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(1) > div:nth-child(1) > a > div.eLAPa > div._9AhH0

In [ ]:
//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[2]/a/div[1]/div[2]
#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(1) > div:nth-child(2) > a > div.eLAPa > div._9AhH0

In [ ]:
//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[3]/a/div/div[2]
#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(1) > div:nth-child(3) > a > div > div._9AhH0

In [ ]:
//*[@id="react-root"]/section/main/article/div[1]/div/div/div[2]/div[1]/a/div/div[2]
#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(2) > div:nth-child(1) > a > div > div._9AhH0

In [ ]:
#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(3) > div:nth-child(1) > a > div.eLAPa > div._9AhH0

In [ ]:
#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(1) > div:nth-child(1) > a > div.eLAPa > div._9AhH0

In [ ]:
#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(1) > div:nth-child(2) > a > div > div._9AhH0
#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(1) > div:nth-child(3) > a > div.eLAPa > div._9AhH0
#react-root > section > main > article > div:nth-child(3) > div > div:nth-child(2) > div:nth-child(1) > a > div > div._9AhH0

In [22]:
temp = driver.find_element_by_css_selector('a > div.eLAPa > div._9AhH0')

In [26]:
source = driver.page_source

In [ ]:
#내용물 클릭할때마다 동일한 속성값을 보유하는듯
body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul > div > li > div > div > div.C4VMK > span
body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul > div > li > div > div > div.C4VMK
body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.EtaWk > ul

In [33]:
soup  = BeautifulSoup(source, 'html.parser')

In [39]:
soup.select('div.KL4Bh')[3]

<div class="KL4Bh"><img alt="Photo by 타이거 슈가 양산점 on September 04, 2020. 이미지: 문구: 'oH! HAPPY 5UGAR DAY 매월 5일 타이거슈가 오! 해피슈가데이 흑당시리즈 3종 500원 할인! 혹당밀크티 4,9007 4,400 혹당크림카페라떼 리얼크림치즈무스혹당밀크티 5,300 5,600 4,800 5,100 TIGER SUGAR 老虎堂黑糖專亮'." class="FFVAD" decoding="auto" sizes="272px" src="https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/118555642_357222802073280_1949621854364072060_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&amp;_nc_cat=100&amp;_nc_ohc=vuVT-WirwFIAX85M6VP&amp;tp=1&amp;oh=abe386e656cf3deb0f368f66c0ca847d&amp;oe=6025D0E3" srcset="https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s150x150/118555642_357222802073280_1949621854364072060_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&amp;_nc_cat=100&amp;_nc_ohc=vuVT-WirwFIAX85M6VP&amp;tp=1&amp;oh=2ec5326e741123f52377743b55637bb7&amp;oe=60262B68 150w,https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s240x240/118555642_357222802073280_1949621854364072060_n.jpg?_nc_ht=scontent-ssn1-1

<html class="js logged-in client-root js-focus-visible sDN5V" lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>#흑당버블티 해시태그 • Instagram 사진 및 동영상</title>
<meta content="noimageindex, noarchive" name="robots"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover" id="viewport" name="viewport"/>
<link href="/data/manifest.json" rel="manifest"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/ConsumerUICommons.css/01aad45fe5a8.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/ConsumerAsyncCommons.css/0608bd6190e0.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/Consumer.css/3466f2ffa63d.css" rel